# GeoMAD Production Notebook

In [ ]:
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray
import os
import pystac

import datacube
from odc.stats.plugins.gm import StatsGMS2
from odc.stats.tasks import TaskReader
from odc.stats.model import OutputProduct
from odc.stac import load
from odc.geo.geobox import GeoBox

### 1. Create Tasks

In [ ]:
# Function to save task with odc-stats
def save_tasks(cloud_cover, output_db):
    dataset_filter = json.dumps({"cloud_cover": [0, cloud_cover]})
    
    !uv run odc-stats save-tasks \
        --frequency "annual" \
        --grid "EPSG:6933;10;5000" \
        --year "2024" \
        --input-products "s2_l2a" \
        --dataset-filter='{{"cloud_cover": [0,{cloud_cover}]}}' \
        {output_db}

In [ ]:
# Execute save_tasks
save_tasks(60, "s2_l2a_2024--P1Y.db")

### 2. Explore Task

In [ ]:
# Load task
grid = gpd.read_file("s2_l2a_2024--P1Y-2024--P1Y.geojson")

In [ ]:
TaskReader("s2_l2a_2024--P1Y.db")

In [ ]:
# Show task interactively, used to pick test tile
grid.explore(
    color="blue",
    style_kwds={"fillOpacity": 0.2, "weight": 0.5, "color": "blue"}
)

### 3. Inspect task database

In [ ]:
# define x, y index and year, which has to be aligned with what's used in odc-stats save-tasks

x, y = 211, -1
year = "2024--P1Y"
cloud_filters= {"cloud shadows":[["dilation", 5]], "cloud medium probability":[["opening", 5], ["dilation", 5]], "cloud high probability":[["opening", 5], ["dilation", 5]], "thin cirrus":[["dilation", 5]]}

In [ ]:
# Dummy product to pass to taskreader
name, version = 'gm_s2', '0-0-1'
product = OutputProduct(name=name,
                        version=version,
                        short_name=name,
                        location = "",
                        properties = {},
                        measurements = [],
                       )

tidx = (year, x, y)
rdr = TaskReader("s2_l2a_2024--P1Y.db", product=product)
task = rdr.load_task(tidx)
task

### 4. Compare datasets in Task to datasets within Database

In [ ]:
# Connect to ODC
dc = datacube.Datacube(app="geomad_s2")

In [ ]:
sentinel2_datasets = dc.find_datasets(
    product=["s2_l2a"],
    time="2024",
    like=task.geobox.to_crs("EPSG:6933"),
    cloud_cover=(0, 60),
)

In [ ]:
# expect same number of datasets found
len(task.datasets), len(sentinel2_datasets)

### 5. Visualise test tile

In [ ]:
# Load available data
ds = dc.load(
    datasets=sentinel2_datasets,
    geopolygon=task.geobox.extent,  
    measurements=["red", "green", "blue", "scl"],
    dask_chunks={"x": 5000, "y": 5000, "time":-1},
    resolution=(-10,10),
    group_by="solar_day",
    output_crs=task.geobox.crs, #"EPSG:6933"
    driver="rio",
)

In [ ]:
task.geobox.explore()

### 6. Process Data with ODC Stats Run

In [ ]:
tile_id = f"{year}/{x:+04d}/{y:+04d}"
base_output_dir = os.path.abspath("./output")
file_url_output = f"file://{base_output_dir}"

In [ ]:
!uv run odc-stats run \
    --plugin="gm-s2" \
    --plugin-config="../config/geomad_config/plugin_config.yaml" \
    --config="../config/geomad_config/main_config.yaml" \
    --threads=5 \
    --memory-limit="40Gi" \
    --location="{file_url_output}" \
    s2_l2a_2024--P1Y.db \
    {tile_id}

### 7. Inspect GeoMAD output

In [ ]:
# Read the file
output_dir = f"{base_output_dir}/x{x}/y{y:03d}/{year}"
tile_name = f"gm_s2_annual_x{x}y{y:03d}_{year}"

In [ ]:
# inspect geobox location
rgba = rioxarray.open_rasterio(f"{output_dir}/{tile_name}_rgba.tif")

rgba.odc.geobox

In [ ]:
# inspect dataset information (ensure COG and CRS)
!gdalinfo {output_dir}/{tile_name}_rgba.tif

In [ ]:
# Check STAC ITEM
stac_item = pystac.Item.from_file(f"{output_dir}/{tile_name}.stac-item.json")

stac_item.to_dict()

In [ ]:
# test load with odc-stac
ds = load([stac_item], bands=["red", "green", "blue"])
rgb = ds.isel(time=0)

# and plot it
rgb[["red", "green", "blue"]].to_array(dim="band").plot.imshow(vmin=0, vmax=3000)
plt.show()

In [ ]:
# check alignment with basemap
rgb.odc.explore(vmin=0, vmax=3000)